## This tests the QED-RHF gradient components against Hilbert's and against numerical components


In [ ]:
from oo_cqed_rhf import CQEDRHFCalculator
import numpy as np
import psi4
psi4.core.be_quiet()

In [ ]:
# Expected gradient contributions for Water in a STO-3G basis set with the following geometry:
h2o_string = """
         O            0.000000000000     0.000000000000    -0.068516219320
         H            0.000000000000    -0.790689573744     0.543701060715
         H            0.000000000000     0.790689573744     0.543701060715
no_reorient
nocom
symmetry c1
"""

# no cavity
lambda_vector = np.array([0, 0, 0.05])
print(lambda_vector)
# options for H2O
psi4_options = {
    "basis": "cc-pVDZ",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-12,
    "d_convergence": 1e-12,
}





In [ ]:
calc = CQEDRHFCalculator(lambda_vector, h2o_string, psi4_options)
calc.calc_cqed_rhf_energy()
_expected_qed_rhf_energy = -76.016355284146
print(F"CQED-RHF ENERGY Matches Expected Value: {np.isclose(calc.cqed_rhf_energy, _expected_qed_rhf_energy, 1e-9,1e-9)}")

In [ ]:
calc.gradient_summary()